# Getting Checkpoints and Tensorboard to work

In [57]:
# Import built-in Python modules
import sys
sys.path.append("..")

# Import third-party modules
import pandas as pd
import numpy as np
import tensorflow as tf
from tf.keras.datasets import mnist
from sklearn.model_selection import train_test_split
import keras_tuner as kt

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.models.build_models import build_model, build_model_kt
from src.models.train import train_model
from src.models.evaluate import evaluate_model, display_tuner_trials
from src.file_management.file_handling import generate_output_dir, load_latest_model
from src.file_management.Logger import Logger
from src.data.preprocess import preprocess_train, preprocess_val_test


MODELS_DIR = Path("../models/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Run in terminal:**
```
tensorboard --logdir "C:\Users\tmane\OneDrive\Documents\GitHub\ml_overview\models" 
```

In [4]:
# tf.data API: https://www.youtube.com/watch?v=4WNz2xrGe8w

(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)

# Create datasets
train_dataset_full = tf.data.Dataset.from_tensor_slices((X_train_full, y_train_full))
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))


train_dataset_full = preprocess_train(train_dataset_full)
train_dataset = preprocess_train(train_dataset)
val_dataset = preprocess_val_test(val_dataset)
test_dataset = preprocess_val_test(test_dataset)

In [5]:
run_dir_path = generate_output_dir("mnist")

Output directory: ..\models\00007-mnist


In [10]:
with Logger(run_dir_path):
    model = build_model()
    train_model(model, train_dataset, val_dataset, run_dir_path, max_epochs=5,  batch_size=128)

Epoch 1/5

Epoch 00001: saving optimizer to ..\models\00007-mnist\checkpoints\model-01-0.20.pickle
750/750 - 7s - loss: 0.3922 - accuracy: 0.8805 - root_mean_squared_error: 5.2277 - val_loss: 0.2022 - val_accuracy: 0.9402 - val_root_mean_squared_error: 5.2497 - 7s/epoch - 9ms/step
Epoch 2/5

Epoch 00002: saving optimizer to ..\models\00007-mnist\checkpoints\model-02-0.16.pickle
750/750 - 3s - loss: 0.1768 - accuracy: 0.9468 - root_mean_squared_error: 5.2289 - val_loss: 0.1586 - val_accuracy: 0.9512 - val_root_mean_squared_error: 5.2500 - 3s/epoch - 5ms/step
Epoch 3/5

Epoch 00003: saving optimizer to ..\models\00007-mnist\checkpoints\model-03-0.15.pickle
750/750 - 3s - loss: 0.1223 - accuracy: 0.9629 - root_mean_squared_error: 5.2291 - val_loss: 0.1539 - val_accuracy: 0.9536 - val_root_mean_squared_error: 5.2501 - 3s/epoch - 4ms/step
Epoch 4/5

Epoch 00004: saving optimizer to ..\models\00007-mnist\checkpoints\model-04-0.15.pickle
750/750 - 3s - loss: 0.0912 - accuracy: 0.9722 - root_m

In [11]:
evaluate_model(model, test_dataset)

Test loss: 0.13705205917358398
Test accuracy: 0.9592000246047974


In [13]:
latest_hdf5, latest_pickle = get_latest_hdf5_pickle(run_dir_path)

epoch, model, opt = load_model_data(latest_hdf5, latest_pickle)

Latest HDF5 file: ..\models\00007-mnist\checkpoints\model-05-0.14.hdf5
Latest pickle file: ..\models\00007-mnist\checkpoints\model-05-0.14.pickle


In [14]:
with Logger(run_dir_path):
    train_model(model, train_dataset, val_dataset, run_dir_path, initial_epoch=epoch, max_epochs=10,  batch_size=128)

Epoch 6/10

Epoch 00006: saving optimizer to ..\models\00007-mnist\checkpoints\model-06-0.15.pickle
750/750 - 4s - loss: 0.0545 - accuracy: 0.9838 - root_mean_squared_error: 5.2295 - val_loss: 0.1542 - val_accuracy: 0.9590 - val_root_mean_squared_error: 5.2503 - 4s/epoch - 5ms/step
Epoch 7/10

Epoch 00007: saving optimizer to ..\models\00007-mnist\checkpoints\model-07-0.13.pickle
750/750 - 3s - loss: 0.0430 - accuracy: 0.9875 - root_mean_squared_error: 5.2295 - val_loss: 0.1299 - val_accuracy: 0.9646 - val_root_mean_squared_error: 5.2504 - 3s/epoch - 5ms/step
Epoch 8/10

Epoch 00008: saving optimizer to ..\models\00007-mnist\checkpoints\model-08-0.15.pickle
750/750 - 4s - loss: 0.0329 - accuracy: 0.9897 - root_mean_squared_error: 5.2296 - val_loss: 0.1482 - val_accuracy: 0.9628 - val_root_mean_squared_error: 5.2504 - 4s/epoch - 5ms/step
Epoch 9/10

Epoch 00009: saving optimizer to ..\models\00007-mnist\checkpoints\model-09-0.14.pickle
750/750 - 4s - loss: 0.0253 - accuracy: 0.9921 - ro

In [15]:
evaluate_model(model, test_dataset)

Test loss: 0.124617800116539
Test accuracy: 0.967199981212616


In [16]:
run_dir = generate_output_dir("mnist_hyp_search")

Output directory: ..\models\00008-mnist_hyp_search


In [30]:
max_epochs = 4
factor = 3
 
# Approx number of models
print("Approx no. epochs per hyperband iteration:", 
      max_epochs * (np.log(max_epochs) / np.log(factor)) ** 2)

Approx no. epochs per hyperband iteration: 6.36915766306784


In [18]:
hyperband_tuner = kt.Hyperband(
    build_model_kt, objective="val_accuracy", seed=42,
    max_epochs=max_epochs, factor=factor, hyperband_iterations=2,
    overwrite=False, directory=MODELS_DIR, project_name=run_dir)


root_logdir = Path(hyperband_tuner.project_dir) / "tb_logs"
tensorboard_cb = tf.keras.callbacks.TensorBoard(str(root_logdir))
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2)
hyperband_tuner.search(train_dataset, epochs=10,
                       validation_data=val_dataset,
                       callbacks=[early_stopping_cb, tensorboard_cb])

Trial 20 Complete [00h 00m 17s]
val_accuracy: 0.9153333306312561

Best val_accuracy So Far: 0.9629999995231628
Total elapsed time: 00h 03m 10s
INFO:tensorflow:Oracle triggered exit


In [53]:
## Get the optimal hyperparameters
# top3_models = hyperband_tuner.get_best_models(num_models=3)
# best_model = top3_models[0]

# display_tuner_trials(hyperband_tuner, 3)

best_hps=hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

{'n_hidden': 2, 'n_neurons': 451, 'learning_rate': 0.001, 'optimizer': 'adam', 'tuner/epochs': 4, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [54]:
run_dir = generate_output_dir("mnist_hyp_search_best")

Output directory: ..\models\00009-mnist_hyp_search_best


In [55]:
# train model using best hyperparameters
with Logger(run_dir):
    best_model = hyperband_tuner.hypermodel.build(best_hps)
    train_model(best_model, train_dataset, val_dataset, run_dir, max_epochs=10,  batch_size=128)

Epoch 1/10

Epoch 00001: saving optimizer to ..\models\00009-mnist_hyp_search_best\checkpoints\model-01-0.20.pickle
750/750 - 5s - loss: 0.3333 - accuracy: 0.8967 - val_loss: 0.1974 - val_accuracy: 0.9370 - 5s/epoch - 7ms/step
Epoch 2/10

Epoch 00002: saving optimizer to ..\models\00009-mnist_hyp_search_best\checkpoints\model-02-0.14.pickle
750/750 - 4s - loss: 0.1454 - accuracy: 0.9554 - val_loss: 0.1444 - val_accuracy: 0.9564 - 4s/epoch - 6ms/step
Epoch 3/10

Epoch 00003: saving optimizer to ..\models\00009-mnist_hyp_search_best\checkpoints\model-03-0.18.pickle
750/750 - 4s - loss: 0.0985 - accuracy: 0.9694 - val_loss: 0.1754 - val_accuracy: 0.9494 - 4s/epoch - 6ms/step
Epoch 4/10

Epoch 00004: saving optimizer to ..\models\00009-mnist_hyp_search_best\checkpoints\model-04-0.14.pickle
750/750 - 5s - loss: 0.0714 - accuracy: 0.9765 - val_loss: 0.1416 - val_accuracy: 0.9617 - 5s/epoch - 6ms/step
Epoch 5/10

Epoch 00005: saving optimizer to ..\models\00009-mnist_hyp_search_best\checkpoin

In [58]:
epoch, model, opt = load_latest_model(run_dir)

Latest HDF5 file: ..\models\00009-mnist_hyp_search_best\checkpoints\model-08-0.15.hdf5
Latest pickle file: ..\models\00009-mnist_hyp_search_best\checkpoints\model-08-0.15.pickle


In [62]:
with Logger(run_dir):
    train_model(model, train_dataset, val_dataset, run_dir, initial_epoch=epoch, max_epochs=12,  batch_size=128)

Epoch 9/12

Epoch 00009: saving optimizer to ..\models\00009-mnist_hyp_search_best\checkpoints\model-09-0.16.pickle
750/750 - 4s - loss: 0.0293 - accuracy: 0.9902 - val_loss: 0.1629 - val_accuracy: 0.9628 - 4s/epoch - 6ms/step
Epoch 10/12

Epoch 00010: saving optimizer to ..\models\00009-mnist_hyp_search_best\checkpoints\model-10-0.16.pickle
750/750 - 4s - loss: 0.0226 - accuracy: 0.9925 - val_loss: 0.1628 - val_accuracy: 0.9655 - 4s/epoch - 6ms/step
Epoch 11/12

Epoch 00011: saving optimizer to ..\models\00009-mnist_hyp_search_best\checkpoints\model-11-0.17.pickle
750/750 - 4s - loss: 0.0235 - accuracy: 0.9920 - val_loss: 0.1719 - val_accuracy: 0.9649 - 4s/epoch - 5ms/step
Epoch 12/12

Epoch 00012: saving optimizer to ..\models\00009-mnist_hyp_search_best\checkpoints\model-12-0.20.pickle
750/750 - 4s - loss: 0.0190 - accuracy: 0.9937 - val_loss: 0.2038 - val_accuracy: 0.9617 - 4s/epoch - 5ms/step
Elapsed time: 0:00:16.49
